In [1]:
#/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  4 10:05:49 2020
@author: kelleclark
"""

import os
import re
import string
import numpy
import matplotlib
import sys
import nltk
#nltk.download('gutenberg')
#nltk.download('genesis')
#nltk.download('inaugural')
#nltk.download('nps_chat')
#nltk.download('webtext')
#nltk.download('treebank')
#from nltk.data import *
#from nltk.book import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
 
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, LSTM
#from keras.utils import np_utils
#from keras.callbacks import ModelCheckpoint

# enumerate ngrams from Eisentein and CSCI6040 ipynb
def ngrams(instring, n):
    outset = {}
    for i in range(len(instring) - n + 1):
        g = ' '.join(instring[i:i+n])
        outset.setdefault(g, 0)
        outset[g] += 1
    return outset

In [2]:
def ngramModel(instring, n):
   outset = tokenize_words(instring)
   totalpossible = len(outset)
   #print (outset)
   anoutcome = ngrams(outset,n)
   if(n != 1):
     previousoutcome = ngrams(outset,n-1)
   #print(anoutcome)
   probmodel = anoutcome
   grams = anoutcome
   test = 0
   for keyword in anoutcome:
        print(f"the word is {keyword} and it occurs {probmodel[keyword]} times")
        if(n==1):
          probmodel[keyword] = (probmodel[keyword] / totalpossible)
          test = test + probmodel[keyword]
        else: 
          listword = keyword.split()
          print(listword[0])
          prob1 = previousoutcome[listword[0]] / totalpossible
          probmodel[keyword] = prob1 *(probmodel[keyword] / previousoutcome[listword[0]])
          test = test + probmodel[keyword]
   print(probmodel)
   print(f"The sum of all he probabiities need to be 1 and it is {test}")
   return probmodel

In [3]:
def oneCorpusFromFiles(folderpath):
    textfiles = [f for f in os.listdir(folderpath) if '.txt' in f]
    corpus = ' '
    for f in textfiles:
        try:
            substring = ' '
            with open(folderpath + '/'+ f) as filetext:
                print(f"the language of file "+f+" is {nltk.language(filetext)}")
                substring = substring.join(line.strip() for line in filetext)
                filetext.close()
                corpus = corpus + substring
        except:
            print("Error reading file:" + f)
    return corpus

In [4]:
def tokenize_words(instring):
    # lowercase and remove punctuation to standardize it
    tokens = word_tokenize(instring)
    tokens = [w.lower() for w in tokens]
    #creating a regular expression for matching
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    punct_free_words = [re_punc.sub('', w) for w in tokens]
    words = [word for word in punct_free_words if word.isalpha()]
    print(f"\n NLTK identified tokenized {len(words)} words.")
    return words

In [5]:
def printoutcorpus(instring,folderpath):
    try:
        if len(instring) > 500: 
            print(folderpath + "\n   First & last 50 characters of Corpus:\n" + instring[:50]+ " . . . "+ instring[-50:])
        elif (len(instring) > 0):
            print(folderpath+f"\n {instring[:]}")
        else:
            print(folderpath + "\n   Corpus is under 100 characters:\n'{corpus}'")
    except:
        print("The corpus does not exist.")


In [6]:
import random
def ngramModel_Smooth(instring, n):
    #generate ngrams
    outset = tokenize_words(instring)
    total = len(outset)
    anoutcome = []
    for i in range(1,n+1):
        print(i)
        anoutcome.append(ngrams(outset, i))
        print("outcome: ")
        print(anoutcome[i-1])
    #generate lamd
    k = 1
    lamd = []
    last_lamd = 0
    for i in range(1,n):
        lamd.append(random.uniform(0,k))
        k = k-lamd[i -1]              
    lamd.append(k)
    print("lamd: ", lamd)
    #generate smooth model
    smooth_model = {}
    for keyword in anoutcome[n-1]:
        grams = keyword.split(' ')
        #print("grams:")
        #print(grams)
        smooth_model.setdefault(keyword, lamd[0]*anoutcome[0][grams[0]]/total)
        for i in range(1,len(grams) - 2):
            sub_string = ' '.join(grams[0:i])
            sub_sub_string = ' '.join(input[0:i -1])
            print(sub_string)
            smooth_model[keyword] = smooth_model[keyword] + lamd[i] * (anoutcome[i][sub_string]/anoutcome[i-1][keyword])
        print(keyword + ":")
        print(smooth_model[keyword])
    print("smooth_model:")
    print(smooth_model)
    return smooth_model

In [7]:
ngramModel_Smooth("This is short test, This is a new doc", 3)


 NLTK identified tokenized 9 words.
1
outcome: 
{'this': 2, 'is': 2, 'short': 1, 'test': 1, 'a': 1, 'new': 1, 'doc': 1}
2
outcome: 
{'this is': 2, 'is short': 1, 'short test': 1, 'test this': 1, 'is a': 1, 'a new': 1, 'new doc': 1}
3
outcome: 
{'this is short': 1, 'is short test': 1, 'short test this': 1, 'test this is': 1, 'this is a': 1, 'is a new': 1, 'a new doc': 1}
lamd:  [0.8323338367213761, 0.06550511845913445, 0.10216104481948948]
this is short:
0.18496307482697247
is short test:
0.18496307482697247
short test this:
0.09248153741348623
test this is:
0.09248153741348623
this is a:
0.18496307482697247
is a new:
0.18496307482697247
a new doc:
0.09248153741348623
smooth_model:
{'this is short': 0.18496307482697247, 'is short test': 0.18496307482697247, 'short test this': 0.09248153741348623, 'test this is': 0.09248153741348623, 'this is a': 0.18496307482697247, 'is a new': 0.18496307482697247, 'a new doc': 0.09248153741348623}


{'this is short': 0.18496307482697247,
 'is short test': 0.18496307482697247,
 'short test this': 0.09248153741348623,
 'test this is': 0.09248153741348623,
 'this is a': 0.18496307482697247,
 'is a new': 0.18496307482697247,
 'a new doc': 0.09248153741348623}

In [21]:
def Generate_text_with_ngrams_model(seedtext):
    pathname = 'Test Data/short test data'
    corp = oneCorpusFromFiles(pathname)
    tokenedcorp = tokenize_words(corp)
    unigrams = ngrams(tokenedcorp,1)
    bigrams = ngrams(tokenedcorp,2)
    trigrams = ngrams(tokenedcorp,3)
    quadgrams = ngrams(tokenedcorp,4)
    r = 10#random.randrange(5,100)
    print(r)
    #using bigram generate r lengh text
    for char in range(0,len(seedtext)):
        maybe = seedtext[:len(seedtext)-char]
        print(f"Searching unigrams for '{maybe}'.") 
        if maybe in unigrams:
            currentword = maybe
            break
        print("Not found. Dropping a letter")
        currentword = "diffenent" 
    start_text = currentword 
    print("Start generate")    
    output_bigrams_model = ''
    output_bigrams_model = output_bigrams_model.join(currentword)
    add_text = ""
    for i in range(1,r):
        high_prob = 0
        sub_grams = dict((key,bigrams[key]) for key in bigrams if key.startswith(currentword + " "))
        for keyword in sub_grams:            
            if sub_grams[keyword] > high_prob:
                add_text = keyword[len(currentword)::]
                high_prob = bigrams[keyword]
        output_bigrams_model = output_bigrams_model + add_text
        currentword = add_text.strip()        
    print("Generate text by bigrams model:")
    print(output_bigrams_model)
    #Generate text by trigrams model
    currentword = start_text
    sub_grams = dict((key,bigrams[key]) for key in bigrams if key.startswith(currentword + " "))
    add_text = ""
    output_trigrams_model = ''
    output_trigrams_model = output_trigrams_model.join(currentword)
    high_prob = 0
    for keyword in sub_grams:
        if sub_grams[keyword] > high_prob:
                add_text = keyword[len(currentword)::]
                high_prob = bigrams[keyword]
    output_trigrams_model = output_trigrams_model + add_text
    currentword = output_trigrams_model;
    for i in range(1,r -1):
        high_prob = 0
        sub_grams = dict((key,trigrams[key]) for key in trigrams if key.startswith(currentword + " "))
        for keyword in sub_grams:            
            if sub_grams[keyword] > high_prob:
                add_text = keyword[len(currentword)::]
                high_prob = trigrams[keyword]
        print("add_text")
        print(add_text)
        output_trigrams_model = output_trigrams_model + add_text
        currentword = currentword.split(" ")[1] + " " + add_text.strip()    
    print("Generate text by trigrams model:")
    print(output_trigrams_model)

In [22]:
Generate_text_with_ngrams_model("for")

the language of file tragedy-of-macbeth.txt is {nltk.language(filetext)}

 NLTK identified tokenized 19803 words.
10
Searching unigrams for 'for'.
Start generate
Generate text by bigrams model:
for a man that i haue done to the time
add_text
 father
add_text
 son
add_text
 nay
add_text
 how
add_text
 will
add_text
 you
add_text
 do
add_text
 not
Generate text by trigrams model:
for a father son nay how will you do not


In [ ]:
#main 
pathname = 'Test Data/short test data'
corp = oneCorpusFromFiles(pathname)
printoutcorpus(corp,pathname)
tokenedcorp = tokenize_words(corp)
print(tokenedcorp[:20])
onegram = ngrams(tokenedcorp, 1)
print(grams)
for k, v in onegram.most_common(3):
   print (k, v)
tot = 0
#onegrammodel = ngramModel(corp, 1)
#print(onegrammodel[:20])
#twogrammodel = ngramModel(corp, 2)